In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
assert os.environ['TPU_NAME'], 'Make sure to select TPU'

In [7]:
## Installing Pytorch/XLA
!pip install cloud-tpu-client==0.10 torch==1.12.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 187.4 MB 25 kB/s s eta 0:00:01
     |████████████████████████████████| 776.3 MB 10 kB/s /s eta 0:00:01    |████████                        | 191.6 MB 3.5 MB/s eta 0:02:46     |██████████████▉                 | 360.0 MB 5.6 MB/s eta 0:01:14��████▉           | 506.5 MB 65.5 MB/s eta 0:00:05
     |████████████████████████████████| 124 kB 42.9 MB/s eta 0:00:01
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.12.0
    Uninstalling absl-py-0.12.0:
      Successfully uninstalled absl-py-0.12.0
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1+cpu
    Uninstalling torch-1.7.1+cpu:
      Successfully uninstalled torch-1.7.1+cpu
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.8.2+cpu requires torch==1.7.1, but you have torch 1.12.0 which is incompatible.
torchtext 0.8

In [8]:
## Creating and manipulating tensors on TPUs.
import torch

import torch_xla
import torch_xla.core.xla_model as xm

In [10]:
#Create a random tensor on xla:1
dev = xm.xla_device()
dev

device(type='xla', index=1)

In [11]:
t1 = torch.ones(3, 3, device = dev)
print(t1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='xla:1')


In [12]:
#Create a tensor on the second cloud TPU core.
second_dev = xm.xla_device(n = 2, devkind = 'TPU')
t2 = torch.ones(3, 3, device = second_dev)
print(t2)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='xla:2')


In [14]:
# Create random filters and inputs to a 1D convolution.
filters = torch.randn(33, 16, 3, device = dev)
inputs = torch.randn(20, 16, 50, device = dev)

torch.nn.functional.conv1d(inputs, filters)

tensor([[[ -0.9738,  -6.6521,   4.9284,  ...,  -5.6360,   2.8959,   1.0662],
         [  5.5568,  -9.0157,   2.3227,  ...,   1.0583,  -4.6777, -14.3902],
         [  2.6976,  -4.6629,  -9.2682,  ..., -11.1429,  16.3411,  -0.9407],
         ...,
         [  1.9748,   4.9036,  -5.9138,  ...,  -1.1551,   4.1300,  -3.2667],
         [ -0.2097,   3.9647,   6.2659,  ...,  -5.1740, -12.0905,  -6.9291],
         [  9.9457,  -4.6346,  -5.1966,  ...,  -1.7209,   7.1797,   4.4891]],

        [[-14.6395,   0.1226,   7.2277,  ..., -10.1350,  -4.0249,  -4.1934],
         [ -4.3426,  11.4777,  -8.2515,  ...,  -3.2779,  -1.4187,   9.2866],
         [ -2.4751,   8.2532,   6.7562,  ...,  -5.0808,   3.5811,  15.6195],
         ...,
         [ -8.8625,  -5.0295, -12.1956,  ..., -10.1487,   2.6805,   4.4668],
         [  6.7460,   0.1803,   0.6393,  ...,  10.1565,   0.2820,   0.1619],
         [  2.4728,  10.7021,   6.8067,  ...,  -2.0000,   2.1387,  -3.8451]],

        [[ 12.8085,  -4.1718,  -0.0709,  ...

In [15]:
## Running PyTorch modules and autograd on TPUs.
#Create a linear module.
fc = torch.nn.Linear(5, 2, bias = True)

#Copy the module to the XLA device.
fc = fc.to(dev)

#Create a random feature tensor.
features = torch.randn(3, 5, device = dev, requires_grad = True)

#Runs and prints the module.
output = fc(features)

print(output)

tensor([[-0.0724,  0.0868],
        [ 0.2145, -0.2787],
        [-0.0324,  0.0093]], device='xla:1', grad_fn=<AddmmBackward0>)


In [16]:
output.backward(torch.ones_like(output))
print(fc.weight.grad)

tensor([[1.1646, 2.1602, 0.4321, 3.1758, 1.9316],
        [1.1646, 2.1602, 0.4321, 3.1758, 1.9316]], device='xla:1')


## Running PyTorch networks on TPUs.

In [20]:
import torch.nn as nn
import torch.nn.functional as F

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        #Perform an affine transformation.
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        #Max pooling over a (2, 2) window.
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] #All dimensions except the batch dimension.
        num_features = 1
        for s in size:
            num_features *= s
            
        return num_features
        

In [22]:
#Places network on the default TPU core.
net = Net().to(dev)

In [23]:
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [25]:
#Creates a random input on the default TPU cores.
inputs = torch.randn(1, 1, 32, 32, device = dev)

#Runs the net.
out = net(inputs)
print(out)

tensor([[ 0.0474, -0.0676,  0.0745,  0.0107, -0.1698, -0.0551, -0.0099,  0.1400,
         -0.1297,  0.0464]], device='xla:1', grad_fn=<AddmmBackward0>)
